### 데이터 준비

## soc 예측

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.io import loadmat

# 데이터 불러오기 함수 정의 (기존과 동일)
def load_data(battery):
    mat = loadmat(f'C:/LEE/batterydata/{battery}.mat')
    dataset = []
    capacity_data = []
    counter = 0

    for i in range(len(mat[battery][0, 0]['cycle'][0])):
        row = mat[battery][0, 0]['cycle'][0, i]
        if row['type'][0] == 'discharge':
            capacity = row['data'][0][0]['Capacity'][0][0]
            for j in range(len(row['data'][0][0]['Voltage_measured'][0])):
                temperature_measured = row['data'][0][0]['Temperature_measured'][0][j]
                current_measured = row['data'][0][0]['Current_measured'][0][j]
                voltage_measured = row['data'][0][0]['Voltage_measured'][0][j]
                dataset.append([counter + 1, temperature_measured, current_measured, voltage_measured, capacity])
            capacity_data.append([counter + 1, capacity])
            counter += 1

    return pd.DataFrame(dataset, columns=['cycle', 'temperature_measured', 'current_measured', 'voltage_measured', 'capacity']), \
           pd.DataFrame(capacity_data, columns=['cycle', 'capacity'])

# 데이터 로드
data_B0005, capacity_B0005 = load_data('B0005')
data_B0006, capacity_B0006 = load_data('B0006')
data_B0007, capacity_B0007 = load_data('B0007')
data_B0018, capacity_B0018 = load_data('B0018')

# 공칭 용량 정의
nominal_capacity = capacity_B0005['capacity'].iloc[0]

# SOC 계산
for data in [data_B0005, data_B0006, data_B0007, data_B0018]:
    data['SOC'] = data['capacity'] / nominal_capacity  # SOC 비율로 계산

### 스케일링 (온도,전류,전압 만)

In [6]:
# Feature scaling (온도, 전류, 전압)
scaler = MinMaxScaler()
train_data = pd.concat([data_B0005, data_B0006, data_B0018], ignore_index=True)
test_data = data_B0007

train_scaled = scaler.fit_transform(train_data[['temperature_measured', 'current_measured', 'voltage_measured']])
test_scaled = scaler.transform(test_data[['temperature_measured', 'current_measured', 'voltage_measured']])

# Sequence preparation for LSTM
def create_sequences(data, soc_values, sequence_length=20):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:i+sequence_length, :]  # Features: 온도, 전류, 전압
        y = soc_values[i + sequence_length]  # SOC 레이블
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 20
X_train, y_train = create_sequences(train_scaled, train_data['SOC'].values, sequence_length)
X_test, y_test = create_sequences(test_scaled, test_data['SOC'].values, sequence_length)

In [9]:
import joblib

joblib.dump(scaler, 'scaler_soc.pkl')

['scaler_soc.pkl']

### LSTM 모델 학습

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# LSTM 모델 정의
model_soc = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)  # Output layer for SOC prediction
])

model_soc.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
history_soc = model_soc.fit(X_train, y_train, epochs=150, batch_size=40, validation_split=0.2, verbose=1)

Epoch 1/150


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2709/2709 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0208 - val_loss: 0.0051
Epoch 2/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0072 - val_loss: 0.0030
Epoch 3/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0068 - val_loss: 0.0051
Epoch 4/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 26s 9ms/step - loss: 0.0066 - val_loss: 0.0076
Epoch 5/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0061 - val_loss: 0.0045
Epoch 6/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0055 - val_loss: 0.0069
Epoch 7/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0046 - val_loss: 0.0049
Epoch 8/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0040 - val_loss: 0.0057
Epoch 9/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0036 - val_loss: 0.0055
Epoch 10/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0034 - val_loss: 0.0086
Epoch 11/150
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0033 - val_loss: 0.0053
Epoch 12/150
2709/27

### 예측 & 평가

In [8]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict SOC on test set
y_pred_soc = model_soc.predict(X_test)

# Calculate RMSE and R^2 for evaluation
rmse_soc = np.sqrt(mean_squared_error(y_test, y_pred_soc))
r2_soc = r2_score(y_test, y_pred_soc)

print(f"Test RMSE for SOC: {rmse_soc}")
print(f"Test R^2 Score for SOC: {r2_soc}")

1571/1571 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
Test RMSE for SOC: 0.1269269555016825
Test R^2 Score for SOC: -1.3815093340269367


In [10]:
model_soc.save('lstm_soc_b000(5,6,7,18).keras')

In [1]:
from tensorflow.keras.models import load_model

# 모델 파일 경로
model_path = 'lstm_soc_b000(5,6,7,18).keras'

# 모델 불러오기
model_soc = load_model(model_path)

# 모델 요약 출력 (옵션)
model_soc.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 50)             │        10,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,555 (127.17 KB)

 Trainable params: 10,851 (42.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 21,704 (84.79 KB)

### 실시간 예측?

In [14]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import joblib
import numpy as np

scaler = joblib.load('scaler_soc.pkl')

def predict_soc_real_time(model, scaler, sequence):
    # Scale sequence
    sequence_scaled = scaler.transform(sequence)
    sequence_scaled = np.expand_dims(sequence_scaled, axis=0)  # Reshape for LSTM
    
    # Predict SOC
    predicted_soc = model.predict(sequence_scaled)[0][0]
    return predicted_soc

# Example usage: Provide a sequence of real-time data
real_time_sequence = [
    [23.7, 0.005, 0.02],  # 가장 오래된 측정값[온도,전류,전압] [23.7, 0.5, 3.7]
    [23.7, 0.005, 0.02],
    [23.7, 0.005, 0.02],
    [23.7, 0.006, 0.02],
    [23.7, 0.004, 0.02],
    [23.7, 0.005, 0.02],
    [23.7, 0.005, 0.02],
    [23.7, 0.005, 0.02],
    [23.7, 0.005, 0.02],
    [23.8, 0.004, 0.02],
    [23.8, 0.006, 0.02],
    [23.8, 0.005, 0.02],
    [23.8, 0.005, 0.02],
    [23.8, 0.004, 0.02],
    [23.8, 0.005, 0.02],
    [23.8, 0.005, 0.02],
    [23.8, 0.006, 0.02],
    [23.8, 0.005, 0.02],
    [23.8, 0.005, 0.02],
    [23.8, 0.005, 0.02]  # 가장 최신 측정값
]
predicted_soc = predict_soc_real_time(model_soc, scaler, real_time_sequence)
print(f"Predicted SOC in real-time: {predicted_soc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted SOC in real-time: 202.5084991455078


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
